In [111]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres
from poliastro.twobody.thrust.change_a_inc import change_a_inc
from poliastro._math.linalg import norm
import poliastro.core.angles as angles
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
# Define the two coplanar debris
a = 6000+6371 << u.km
ecc = 0 << u.one
inc = 45 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 0 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 6000+6371 << u.km
inc = 45 << u.deg
nu = 45 << u.deg
raan = -75 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [129]:
import CustomManeuvres
raan_change = CustomManeuvres.simple_raan_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = raan_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(raan_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)


theta 50.9934283953222 deg
u_final 61.516533882068266 deg
thrust_norm -4886.845514211979 m / s
y_thrust 2103.5882682276965 m / s
z_thrust 4410.9154919980665 m / s
nu:  1.0736660606567405 rad
-0.8789547700971942
-0.47690513954390185
otv inc 44.999999999999986 deg
deb_2 inc 45.0 deg


In [74]:
otv.argp

<Quantity 0. deg>

In [46]:
import CustomManeuvres
inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

thrust_location mean:  179.999 deg
thrust_norm 3922.362414636926 m / s
y_thrust 1179.477125838908 m / s
z_thrust -3740.8235218169816 m / s
nu:  3.1415752002972717 rad
-1.7453292520688482e-05
0.9999999998476913
otv inc 44.99999999514993 deg
deb_2 inc 45.0 deg


In [46]:
nu = 3.1415752002972717 * u.rad
nu
np.cos(nu)

<Quantity -1.>

In [13]:
print(deb_1.nu << u.deg)
print(otv.nu << u.deg)
print(deb_2.nu << u.deg)
print((otv.nu + otv.argp) << u.deg)

# mean_anomaly = angles.nu_to_M(otv.nu, otv.ecc)

# print(mean_anomaly << u.deg)

179.9989999999999 deg
89.99918085912401 deg
172.07740403789592 deg
179.99975442439205 deg


In [47]:
# Propapage all a bit
t = 1000 * u.s
deb_1 = deb_1.propagate(t)
deb_2 = deb_2.propagate(t)
otv = otv.propagate(t)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [33]:
otv.argp << u.deg

<Quantity 90.00057357 deg>

In [48]:
import CustomManeuvres


inc_change = CustomManeuvres.simple_inc_change(otv, deb_1)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_1 inc', deb_1.inc << u.deg)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

thrust_location mean:  0.0 deg
thrust_norm -3922.3624126801215 m / s
y_thrust 1179.4771250921556 m / s
z_thrust 3740.823520000665 m / s
nu:  -1.5708429495477318 rad
0.9999999989131595
4.662275281833854e-05
zero deg transformation
otv inc 10.00000003266459 deg
deb_1 inc 10.0 deg


In [42]:
inc_change.impulses

((<Quantity 3513.9909823 s>,
  <Quantity [ 1179.51457417,  2645.1533294 , -2645.15333027] m / s>),)

In [19]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas

/Users/hugoderohanwillner/Documents/MSc AI/Lab Project/Poliastro_Testing/CustomLowLevel.py:49: NumbaPerformanceWarning:

'@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))



Target Delta:  31.72445180004123 deg
current_delta :  352.07764961350387 deg
dist:  320.35319781346266 deg
t_1:  23578.832486387666 s
mean_anomaly_i:  40.23086226662301 deg
new delta:  31.72508963397705 deg


Number of impulses: 2, Total cost: 0.841563 km / s

In [20]:
hoh_phas.impulses

((<Quantity 23578.83248639 s>,
  <Quantity [-281.21813403,  235.05850625,  235.05452489] m / s>),
 (<Quantity 5640.07210305 s>,
  <Quantity [ 262.31467383, -219.25789249, -219.25417875] m / s>))

In [21]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [18]:
# Wrong values, need to use mean anomaly to compare
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))
print('difference: ', (otv.nu - deb_2.nu).to(u.deg))
print('OTV.a: ', otv.a)
print('Deb2.a: ', deb_2.a)

OTV ANGLE:  -172.61536114641075 deg
DEB2 ANGLE:  157.19332482068768 deg
difference:  -329.8086859670984 deg
OTV.a:  9371.0 km
Deb2.a:  9371.0 km
